In [ ]:
# -------------------------------
# 0. Safety for Jupyter / M2 macOS
# -------------------------------
import matplotlib
matplotlib.use("Agg")  # avoids crashes in Jupyter on macOS M2

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os


In [ ]:
# -------------------------------
# 1. Generate synthetic regression data
# -------------------------------
np.random.seed(42)  # for reproducibility
X = np.linspace(0, 10, 300).reshape(-1, 1).astype(np.float32)
y = 2 * X + 3 + np.random.randn(*X.shape) * 0.5

In [ ]:
# -------------------------------
# 2. Build the model (MLP)
# -------------------------------
model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,)),           # Recommended over input_shape in Dense
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='mse',
    metrics=['mae']   # Mean Absolute Error as metric
)

In [ ]:
# -------------------------------
# 3. Callbacks
# -------------------------------
early_stop = tf.keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True
)

checkpoint_path = "best_model.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    save_best_only=True
)

In [ ]:

# -------------------------------
# 4. Train the model
# -------------------------------
history = model.fit(
    X, y,
    epochs=100,
    validation_split=0.2,
    callbacks=[early_stop, checkpoint],
    verbose=1
)


In [ ]:

# -------------------------------
# 5. Load best weights (optional)
# -------------------------------
model.load_weights(checkpoint_path)

In [ ]:

# -------------------------------
# 6. Evaluate final model
# -------------------------------
loss, mae = model.evaluate(X, y, verbose=0)
print(f"Final MAE: {mae:.3f}")
final_val_mae = history.history['val_mae'][-1]
print(f"Final Validation MAE: {final_val_mae:.3f}")

In [ ]:
# -------------------------------
# 7. Predict and visualize
# -------------------------------
preds = model.predict(X)

plt.figure(figsize=(8,5))
plt.scatter(X, y, s=10, label="True Data")
plt.plot(X, preds, color='red', label="Predicted")
plt.xlabel("X")
plt.ylabel("y")
plt.title("MLP Regression Predictions")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# -------------------------------
# 8. Plot training history
# -------------------------------
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Training Loss (MSE)')
plt.plot(history.history['val_loss'], label='Validation Loss (MSE)')
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.title("Training History")
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(8,4))
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.xlabel("Epoch")
plt.ylabel("MAE")
plt.title("MAE History")
plt.legend()
plt.grid(True)
plt.show()
